In [22]:
import networkx as nx
import numpy as np
from cvxopt import matrix, solvers
from tqdm.auto import tqdm

from gc4eptn.utils import utils
from gc4eptn.utils import metrics
from gc4eptn import gsp
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Example 


$$
\begin{split}\begin{array}{ll}
\mbox{minimize}  &  2x_1^2 + x_2^2 + x_1 x_2 + x_1 + x_2 \\
\mbox{subject to} & x_1 \geq 0 \\
       & x_2 \geq 0 \\
       & x_1  + x_2  = 1
\end{array}\end{split}
$$

QP Generalization

$$
1/2 x^\top P x + q^\top x
$$

$P \in (n \times n)$ where $n$ is the number of variable and values represent quadratic matrix

$q \in (n \times 1)$ where $n$ is the number of variable and values represent affine vector

$G \in (n \times n)$ is the inequality coefficient for $\leq$. Multiply by -1 to turn $\geq$ into $\leq$. Rows correspond equations.

$h \in (m \times 1)$ is the right-hand side of the inequality equation, where $m$ is the number of inequality equations. Rows correspond equations.

$A \in (1 \times b)$ is the coefficient of the equality equations, where $b$ is the number of equality equations.

$b \in (b \times 1)$ is the right-hand side of the equality equation, where $b$ is the number of equality equations. Rows correspond equations.

In [2]:
P = 2*matrix([ [2, .5], [.5, 1] ]) # quadratic terms 1/2 x^T P x so P is

q = matrix([1.0, 1.0])

G = matrix([[-1.0,0.0],[0.0,-1.0]])

h = matrix([0.0,0.0])

A = matrix([1.0, 1.0], (1,2))

b = matrix(1.0)

In [3]:
sol=solvers.qp(P, q, G, h, A, b)

     pcost       dcost       gap    pres   dres
 0:  1.8889e+00  7.7778e-01  1e+00  2e-16  2e+00
 1:  1.8769e+00  1.8320e+00  4e-02  1e-16  6e-02
 2:  1.8750e+00  1.8739e+00  1e-03  2e-16  5e-04
 3:  1.8750e+00  1.8750e+00  1e-05  1e-16  5e-06
 4:  1.8750e+00  1.8750e+00  1e-07  3e-16  5e-08
Optimal solution found.


# GSP Synthetic Data Tests

https://github.com/Anou9531/Laplacian/blob/master/code/main.py

In [18]:
class synthetic_data_gen:
    def __init__(self, num_vertices=20):
        self.num_vertices = num_vertices
        self.constants()
        self.create_graphs()

    def constants(self):
        self.alpha_rnd = 0.012
        self.beta_rnd = 0.79
        self.thr_rnd = 0.06

        self.alpha_er = 0.0032
        self.beta_er = 0.10
        self.thr_er = 0.10

        self.alpha_ba = 0.0025
        self.beta_ba = 0.05
        self.thr_ba = 0.46

    def create_graphs(self):
        self.create_er_graph()
        self.create_ba_graph()
        self.create_random_graph()
        return

    def create_er_graph(self):
        self.er_prob = 0.2
        self.er_graph = nx.erdos_renyi_graph(self.num_vertices, self.er_prob)
        self.er_normL = nx.normalized_laplacian_matrix(self.er_graph)
        return

    def create_ba_graph(self):
        self.ba_graph = nx.barabasi_albert_graph(self.num_vertices, 1)
        self.ba_normL = nx.normalized_laplacian_matrix(self.ba_graph)

    def create_random_graph(self):
        self.random_graph = nx.random_geometric_graph(self.num_vertices, 0.4)
        for u, v, d in self.random_graph.edges(data=True):
            pos1 = np.array(self.random_graph._node[u]['pos'])
            pos2 = np.array(self.random_graph._node[v]['pos'])
            d['weight'] = np.exp(-np.linalg.norm(pos1 - pos2) / (2 * 0.5 * 0.5))
        self.rg_normL = nx.normalized_laplacian_matrix(self.random_graph)

    def get_gs(self, graphL, num_sigs):
        D, V = np.linalg.eig(graphL)
        idx = D.argsort()[::-1]
        D = D[idx]
        V = V[:, idx]
        sigma = np.linalg.pinv(np.diag(D))
        mu = np.zeros(D.shape[0])
        gs_coeff = np.random.multivariate_normal(mu, sigma, num_sigs)
        gs = np.dot(V, gs_coeff.T)
        gs += 0.5 * np.random.randn(*gs.shape)
        # Shape of gs is num_nodes x num_sigs
        # Output has each row as a signal
        return gs.T

    def get_graph_signals(self):
        # Each row is supposed to be a signal
        graph_signals_er = self.get_gs(self.er_normL.toarray(), 100)
        graph_signals_ba = self.get_gs(self.ba_normL.toarray(), 100)
        graph_signals_rand = self.get_gs(self.rg_normL.toarray(), 100)
        return (graph_signals_er, graph_signals_ba, graph_signals_rand)

In [19]:
def L_graph_precision(A, L, thresh=1e-4):
    A_hat = utils.to_adjacency(utils.thresholding(L, thresh)) 
    return metrics.graph_precision(A=A, A_hat=A_hat)

def L_graph_recall(A, L, thresh=1e-4):
    A_hat = utils.to_adjacency(utils.thresholding(L, thresh)) 
    return metrics.graph_precision(A_hat=A, A=A_hat)

def get_f_score(prec, recall):
    return 2 * prec * recall / (prec + recall)

def get_MSE(L_out, L_gt):
    return np.linalg.norm(L_out - L_gt, 'fro')

In [20]:
a1 = synthetic_data_gen()
g1, g2, g3 = a1.get_graph_signals()

In [23]:
# np.random.seed(0)
solvers.options['show_progress'] = False
syn = synthetic_data_gen()
num_nodes = syn.num_vertices

prec_er_list = []
prec_ba_list = []
prec_rnd_list = []

recall_er_list = []
recall_ba_list = []
recall_rnd_list = []

f_score_er_list = []
f_score_ba_list = []
f_score_rnd_list = []

mse_rnd_list = []
max_iter = 50
iters = range(10)
for i in tqdm(iters):
    # np.random.seed(i)
    graph_signals_er, graph_signals_ba, graph_signals_rand = syn.get_graph_signals()
    L_er, Y_er = gsp.gl_sig_model(graph_signals_er, max_iter, syn.alpha_er, syn.beta_er)
    L_ba, Y_ba = gsp.gl_sig_model(graph_signals_ba, max_iter, syn.alpha_er, syn.beta_er)
    L_rnd, Y_rnd = gsp.gl_sig_model(graph_signals_rand, max_iter, syn.alpha_rnd, syn.beta_rnd)

    L_er_gt = nx.laplacian_matrix(syn.er_graph)
    L_ba_gt = nx.laplacian_matrix(syn.ba_graph)
    L_rnd_gt = nx.laplacian_matrix(syn.random_graph)
                                  
    prec_er = L_graph_precision(L=L_er, A=-L_er_gt.todense(), thresh=syn.thr_er)
    prec_ba = L_graph_precision(L=L_ba, A=-L_ba_gt.todense(), thresh=syn.thr_ba)
    prec_rnd = L_graph_precision(L=L_rnd, A=utils.to_adjacency(-L_rnd_gt.todense()), thresh=syn.thr_rnd)

    recall_er = L_graph_recall(L=L_er, A=-L_er_gt.todense(), thresh=syn.thr_er)
    recall_ba = L_graph_recall(L=L_ba, A=-L_ba_gt.todense(), thresh=syn.thr_ba)
    recall_rnd = L_graph_recall(L=L_rnd, A=utils.to_adjacency(-L_rnd_gt.todense()), thresh=syn.thr_rnd)
    
    mse_rnd_list.append(get_MSE(L_rnd, L_rnd_gt))

    prec_er_list.append(prec_er)
    recall_er_list.append(recall_er)
    f_score_er_list.append(get_f_score(prec_er, recall_er))

    prec_ba_list.append(prec_ba)
    recall_ba_list.append(recall_ba)
    f_score_ba_list.append(get_f_score(prec_ba, recall_ba))

    prec_rnd_list.append(prec_rnd)
    recall_rnd_list.append(recall_rnd)
    f_score_rnd_list.append(get_f_score(prec_rnd, recall_rnd))

print('Avg F-score RBF', np.mean(f_score_rnd_list))
print('Avg Prec RBF', np.mean(prec_rnd_list))
print('Avg Recall RBF', np.mean(recall_rnd_list))
print("="*50)
print('Avg F-score ER', np.mean(f_score_er_list))
print('Avg Prec ER', np.mean(prec_er_list))
print('Avg Recall ER', np.mean(recall_er_list))
print("="*50)
print('Avg F-score BA', np.mean(f_score_ba_list))
print('Avg Prec BA', np.mean(prec_ba_list))
print('Avg Recall BA', np.mean(recall_ba_list))
print("="*50)
print('Avg MSE Rnd', np.mean(mse_rnd_list))

  0%|          | 0/10 [00:00<?, ?it/s]

Avg F-score RBF 0.7620780102833729
Avg Prec RBF 0.8750202592682506
Avg Recall RBF 0.6766233766233766
Avg F-score ER 0.0
Avg Prec ER -0.5658823348297032
Avg Recall ER 0.0
Avg F-score BA 0.0
Avg Prec BA -1.4005555555555556
Avg Recall BA 0.0
Avg MSE Rnd 19.93564250864741
